Importing Libraries

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

import glob
import pprint

pp = pprint.PrettyPrinter()

Connecting Google Drive

In [ ]:
#Mount Google Drive in Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Setting a Directory for Cache in Google Drive

import os

cache_dir = '/content/drive/MyDrive/transformers_cache'
os.makedirs(cache_dir, exist_ok=True)

Loading T5 Model and Tokenizer and chaching them in Google Drive

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', cache_dir=cache_dir)
model = T5ForConditionalGeneration.from_pretrained('t5-base', cache_dir=cache_dir)

Defining a summarize_text function

In [ ]:
def summarize_text(text, model, tokenizer, min_length=50, max_length=150, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=512,  # Keep this high to handle long inputs
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        min_length=min_length,            # Minimum length to avoid too short a summary
        max_length=max_length,            # Adjust this to control summary length
        num_beams=num_beams,
        length_penalty=2.0,               # Higher values encourage longer summaries
        no_repeat_ngram_size=3,           # Prevent repetitive phrases
        early_stopping=True               # Stops generating when the model thinks it's done
    )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Calculate the word count of the summary
    word_count = len(summary.split())

    return summary, word_count

Calling the function and generating a summary

In [ ]:
text = """
The global smartphone market experienced a significant shift in 2023,
with total shipments reaching 1.21 billion units, a decline of 7.1% compared to the previous year.
This was primarily due to supply chain disruptions, inflationary pressures, and geopolitical tensions.
In contrast, Apple's market share grew from 23.4% in 2022 to 26.5% in 2023, driven by strong demand for the iPhone 14 series,
particularly in North America and Europe. Meanwhile, Samsung's market share slightly decreased from 19.1% to 18.7%,
despite launching the Galaxy S23 and Galaxy Z Fold 5. Xiaomi retained its third position globally,
with an 11.8% market share, but also saw a 10% drop in shipments, largely due to a sluggish market in India.
Additionally, 5G-enabled devices accounted for 70% of total smartphone shipments, up from 57% in 2022,
reflecting increased consumer demand for faster mobile internet speeds. Analysts predict a rebound in 2024,
with smartphone shipments expected to grow by 4.3%, as supply chain constraints ease and
new innovations such as foldable screens and AI-powered features become more mainstream.
"""

# Call the function and generate the summary
summary = summarize_text(text, model, tokenizer)

# Print the summary
print("Summary:")
print(summary)

Summary:
('the global smartphone market experienced a significant shift in 2023. total shipments reached 1.21 billion units, a decline of 7.1% compared to the previous year. analysts predict a rebound in 2024, with smartphone shipments expected to grow by 4.3%.', 39)


**Rouge Score**

Installing libraries

In [1]:
!pip install datasets rouge-score

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=11e40a7c29b29000f3d3d2120dd6991b4f83c93552413a7a55b1be50701dc516
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


Calculating Rouge Score

In [4]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00


In [10]:
import evaluate

# Initialize the ROUGE metric
rouge = evaluate.load("rouge")

# Example data: original summaries and generated summaries
original_summaries = [
    '''
The global smartphone market experienced a significant shift in 2023,
with total shipments reaching 1.21 billion units, a decline of 7.1% compared to the previous year.
This was primarily due to supply chain disruptions, inflationary pressures, and geopolitical tensions.
In contrast, Apple's market share grew from 23.4% in 2022 to 26.5% in 2023, driven by strong demand for the iPhone 14 series,
particularly in North America and Europe. Meanwhile, Samsung's market share slightly decreased from 19.1% to 18.7%,
despite launching the Galaxy S23 and Galaxy Z Fold 5. Xiaomi retained its third position globally,
with an 11.8% market share, but also saw a 10% drop in shipments, largely due to a sluggish market in India.
Additionally, 5G-enabled devices accounted for 70% of total smartphone shipments, up from 57% in 2022,
reflecting increased consumer demand for faster mobile internet speeds. Analysts predict a rebound in 2024,
with smartphone shipments expected to grow by 4.3%, as supply chain constraints ease and
new innovations such as foldable screens and AI-powered features become more mainstream.

'''
]

generated_summaries = [
    '''
    the global smartphone market experienced a significant shift in 2023.
    total shipments reached 1.21 billion units, a decline of 7.1% compared to the previous year.
    analysts predict a rebound in 2024, with smartphone shipments expected to grow by 4.3%.

    '''
]

# Calculate ROUGE scores
results = rouge.compute(predictions=generated_summaries, references=original_summaries)

# Display the results
print("ROUGE-1:", results["rouge1"])
print("ROUGE-2:", results["rouge2"])
print("ROUGE-L:", results["rougeL"])

ROUGE-1: 0.36444444444444446
ROUGE-2: 0.33183856502242154
ROUGE-L: 0.36444444444444446
